# Quadratic Assignment Problem

#### Problem Description

The goal of the Quadratic Assignment Problem (QAP) is to assign 𝑁 facilities to 𝑁 locations so that assignment costs are minimized.
The assignment costs are defined by a function that multiplies and summarizes the flow/demand (f) and distance (d) for each assignment

In [ ]:
from dwave_qbsolv import QBSolv
from dwave.system import FixedEmbeddingComposite, DWaveSampler
import dwave.inspector
from dimod import qubo_to_ising
from minorminer import minorminer
import networkx as nx
from FileParser import FileParser
import string
import numpy as np

#### Setup Problem

In [ ]:
path_to_file = "./"
file_name = "test_instance_5.dat"
file_parser = FileParser(path_to_file, file_name)
flows, distances, num_nodes = file_parser.parse_file()

print("Distances:\n", distances)
print("Flows: \n", flows)
print("Number of nodes:", num_nodes)

In [ ]:
# Facility to location mapping. Used for decoding quantum annealing response
facility_to_location = []
for i in range(num_nodes):
    for j in range(num_nodes):
        facility_to_location.append(string.ascii_uppercase[i] + str(j))
print("Facility to location:", facility_to_location)

#### Build QUBO

In [ ]:
length_of_QUBO = num_nodes**2
prefactor = distances.max() * flows.max() + 1
QUBO = {}

In [ ]:
# Fill upper triangular QUBO Matrix
for v in range(0, length_of_QUBO):
    for j in range(v, length_of_QUBO):
        QUBO[(v, j)] = 0.0

In [ ]:
# Optimization Function (add distances and flows)
# Add distances and flows
dist_x = 0
dist_y = 0
for v in range(0, length_of_QUBO):
    for j in range(v, length_of_QUBO):
        if j % num_nodes == 0 and v != j:
            dist_y += 1
        if v % num_nodes == 0 and j == v and v != 0:
            dist_x += 1
        QUBO[(v, j)] = QUBO[(v, j)] + distances[dist_x][dist_y] * flows[v % num_nodes][j % num_nodes]

        if j == length_of_QUBO-1:
            dist_y = dist_x
            if v % num_nodes == num_nodes-1:
                dist_y += 1

In [ ]:
# Constraint that every location has exactly one facility assigned
for v in range(0, length_of_QUBO):
    for j in range(v, length_of_QUBO):
        if v == j:
            QUBO[(v, j)] = QUBO[(v, j)] + (-1.0) * prefactor
        else:
            if j % num_nodes == v % num_nodes:
                QUBO[(v, j)] = QUBO[(v, j)] + 2.0 * prefactor

In [ ]:
# Constraint that every facility has exactly one location assigned
for v in range(0, length_of_QUBO):
    for j in range(v, length_of_QUBO):
        if j % num_nodes == 0 and v != j:
            break
        if v == j:
            QUBO[(v, j)] = QUBO[(v, j)] + (-1.0) * prefactor
        else:
            QUBO[(v, j)] = QUBO[(v, j)] + 2.0 * prefactor

qubo = {k:v for k,v in QUBO.items() if v != 0.0 or k[0] == k[1]}            

#### Set Execution Parameters

In [ ]:
# QBSolv or D-Wave Parameter
use_d_wave = True

if not use_d_wave:
    # QbSolv Parameter
    num_repeats = 50
    solver = 'tabu'
    
else:
    # D-Wave Parameter
    solver_limit = num_nodes**2
    solver = 'solver'
    token = 'token'
    endpoint = 'url'
    num_reads = 1000
    chain_strength = (max(qubo.values()) + 1)

#### Run Problem

In [ ]:
# Call QBSolv or D-Wave
if not use_d_wave:
    answer = QBSolv().sample_qubo(qubo, num_repeats=num_repeats, solver=solver)
else:
    G = nx.complete_graph(solver_limit)
    dw_solver = DWaveSampler(solver=solver, token=token, endpoint=endpoint)
    embedding = minorminer.find_embedding(G.edges, dw_solver.edgelist)
    # print("Embedding", embedding)
    
    h, J, offset = qubo_to_ising(qubo)
    answer = FixedEmbeddingComposite(dw_solver, embedding).sample_ising(h, J, num_reads=num_reads, chain_strength=chain_strength)
    

#### Decode Answer

In [ ]:
sample_information = []
graph_labels = {}
for sample, energy, occ in answer.record:
    sample_information.append([sample.tolist(), energy, occ])
sample_information.sort(key=lambda x: x[:][1])

# Decode answer
for i, sample in enumerate(sample_information):
    print(sample)
    for j in range(len(sample[0])):
        if sample[0][j] == 1:
            print(facility_to_location[j])
            if i == 0:
                graph_labels.update({int(j%num_nodes):facility_to_location[j]})
# dwave.inspector.show(answer)